In [1]:
#import libraries
import numpy as np
import tensorflow as tf
import sys as sy
import struct as st

In [2]:
#input data
fpI = open('/home2/bamboo/local_workspace/workspace/Out/barbara512/ewpLH-ImgRes_0.dig', 'rb')
nxI = fpI.read(4)
nxI = st.unpack("i", nxI)
nxI = nxI[0]
nyI = fpI.read(4)
nyI = st.unpack("i", nyI)
nyI = nyI[0]
img_target = []
for _ in range(nxI * nyI):
    data = fpI.read(8)
    data = st.unpack("d", data)
    data = data[0]
    img_target.append(data)
fpI.close()
img_target = np.array(img_target)
target = img_target.reshape(nyI, nxI)
fpIL = open('/home2/bamboo/local_workspace/workspace/Out/barbara512/ewpLH-LImgRes_0.dig', 'rb')
nxIL = fpIL.read(4)
nxIL = st.unpack("i", nxIL)
nxIL = nxIL[0]
nyIL = fpIL.read(4)
nyIL = st.unpack("i", nyIL)
nyIL = nyIL[0]
img_low = []
for _ in range(nxIL * nyIL):
    data = fpIL.read(8)
    data = st.unpack("d", data)
    data = data[0]
    img_low.append(data)
fpIL.close()
img_low = np.array(img_low)
low = img_low.reshape(nyIL, nxIL)

In [3]:
# Define the size of extract block
width = nxI
height = nyI
wid_block = 10
hei_block = 10
num_target = 2
num_blocks = (width - (wid_block - num_target)) * (height - (hei_block - 1))
init_target = 2 * height + 2
half_widblock = int(wid_block * 0.5)
dim_block = int(wid_block * hei_block)
wid_target_block = wid_block
hei_target_block = hei_block - 1
dim_target_block = wid_target_block * hei_target_block
wid_low_block = wid_block
hei_low_block = 1
dim_low_block = wid_low_block * hei_low_block

In [4]:
#make dataset
arr_data = []
for y in range(height - hei_block + 1):
    for x in range(0, width - wid_block + 1, 2):
        target_block = target[y:y+hei_target_block, x:x+wid_target_block]
        target_block = target_block.reshape(dim_target_block, 1)
        arr_data.extend(target_block)
        low_block = low[y + 2, x:x+wid_low_block]
        low_block = low_block.reshape(dim_low_block, 1)
        arr_data.extend(low_block)
train_data = np.array(arr_data)
train_x = train_data.reshape(int(train_data.size / dim_block), dim_block)
d = []
for y in range(hei_block - 1, height, 1):
    for x in range(half_widblock - 1, width - (half_widblock - 1), 2):
        label = target[y, x:x+2]
        d.append(label)
label = np.array(d)
train_label = np.reshape(label, (len(train_x), 2))

In [55]:
train_x.shape

(62244, 100)

In [5]:
#define num
num_input_units = dim_block
num_hidden_units = 68
# num_hidden2_units = 
# num_hidden3_units = 28
# num_hidden4_units = 28
# num_hidden5_units = 28
learning_rate = 0.0001
num_iter = 5000
num_output = 2
train_size = train_x.shape[0]
batch_size = 32

In [6]:
#set parameters, output is 2
x = tf.placeholder(tf.float32, [None, num_input_units])
wscale = 2.0
sigma1 = tf.sqrt(1.0/num_input_units * wscale)
sigma2 = tf.sqrt(1.0/num_hidden_units * wscale)
# sigma3 = tf.sqrt(1.0/num_hidden2_units * wscale)
# sigma4 = tf.sqrt(1.0/num_hidden3_units * wscale)
# sigma5 = tf.sqrt(1.0/num_hidden4_units * wscale)
# sigma6 = tf.sqrt(1.0/num_hidden5_units * wscale)

#hidden layer 1
w_h_1 = tf.Variable(tf.random_normal(shape=[num_input_units, num_hidden_units], stddev=sigma1))
b_h_1 = tf.Variable(tf.zeros([num_hidden_units]))

#hidden layer2
w_h_2 = tf.Variable(tf.random_normal(shape=[num_hidden_units, num_hidden_units], stddev=sigma2))
b_h_2 = tf.Variable(tf.zeros([num_hidden_units]))

# #hidden layer3
# w_h_3 = tf.Variable(tf.random_normal(shape=[num_hidden2_units, num_hidden3_units], stddev=sigma3))
# b_h_3 = tf.Variable(tf.zeros([num_hidden3_units]))

# #hidden layer4
# w_h_4 = tf.Variable(tf.random_normal(shape=[num_hidden3_units, num_hidden4_units], stddev=sigma4))
# b_h_4 = tf.Variable(tf.zeros([num_hidden4_units]))

# #hidden layer5
# w_h_5 = tf.Variable(tf.random_normal(shape=[num_hidden4_units, num_hidden5_units], stddev=sigma5))
# b_h_5 = tf.Variable(tf.zeros([num_hidden5_units]))

#output layer
w_o = tf.Variable(tf.random_normal(shape=[num_hidden_units, num_output], stddev=sigma2))
b_o = tf.Variable(tf.zeros([num_output]))

#calculate hidden layer
h_1 = tf.nn.relu(tf.matmul(x, w_h_1) + b_h_1)
h_2 = tf.nn.relu(tf.matmul(h_1, w_h_2) + b_h_2)
# h_3 = tf.nn.relu(tf.matmul(h_2, w_h_3) + b_h_3)
# h_4 = tf.nn.relu(tf.matmul(h_3, w_h_4) + b_h_4)
# h_5 = tf.nn.relu(tf.matmul(h_4, w_h_5) + b_h_5)

#calculate output layer
y = tf.matmul(h_2, w_o) + b_o

t = tf.placeholder(tf.float32, [None, num_output])


loss = tf.losses.mean_squared_error(y, t)
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

LOSS = sess.run(tf.sqrt(loss), feed_dict={x:train_x, t:train_label})

In [7]:
# start iterating
for i in range(num_iter + 1):
#     batch_xs, batch_ts = train_data.train.next_batch(100)
    batch_mask = np.random.choice(train_size, batch_size)
    batch_x = train_x[batch_mask]
    batch_label = train_label[batch_mask]
    sess.run(train_step, feed_dict={x:batch_x, t:batch_label})
    loss_new = sess.run(tf.sqrt(loss), feed_dict={x:batch_x, t:batch_label})
    if LOSS > loss_new:
        LOSS = loss_new
        weight_1 = sess.run(w_h_1)
        weight_2 = sess.run(w_h_2)
#         weight_3 = sess.run(w_h_3)
#         weight_4 = sess.run(w_h_4)
#         weight_5 = sess.run(w_h_5)
        weight_out = sess.run(w_o)
        bias_1 = sess.run(b_h_1)
        bias_2 = sess.run(b_h_2)
#         bias_3 = sess.run(b_h_3)
#         bias_4 = sess.run(b_h_4)
#         bias_5 = sess.run(b_h_5)
        bias_out = sess.run(b_o)
    if(i % 100 == 0):
        print('Step: %d, Loss: %f' % (i, LOSS))
print('END')

Step: 0, Loss: 117.174957
Step: 100, Loss: 21.775331
Step: 200, Loss: 18.078091
Step: 300, Loss: 15.266818
Step: 400, Loss: 15.266818
Step: 500, Loss: 13.902082
Step: 600, Loss: 13.902082
Step: 700, Loss: 13.902082
Step: 800, Loss: 13.902082
Step: 900, Loss: 13.902082
Step: 1000, Loss: 13.902082
Step: 1100, Loss: 13.902082
Step: 1200, Loss: 13.902082
Step: 1300, Loss: 13.902082
Step: 1400, Loss: 13.698524
Step: 1500, Loss: 13.100378
Step: 1600, Loss: 13.100378
Step: 1700, Loss: 13.100378
Step: 1800, Loss: 13.100378
Step: 1900, Loss: 13.100378
Step: 2000, Loss: 12.609237
Step: 2100, Loss: 12.609237
Step: 2200, Loss: 12.609237
Step: 2300, Loss: 12.609237
Step: 2400, Loss: 12.609237
Step: 2500, Loss: 12.609237
Step: 2600, Loss: 12.609237
Step: 2700, Loss: 10.935732
Step: 2800, Loss: 10.935732
Step: 2900, Loss: 10.935732
Step: 3000, Loss: 10.935732
Step: 3100, Loss: 10.935732
Step: 3200, Loss: 10.935732
Step: 3300, Loss: 10.935732
Step: 3400, Loss: 10.935732
Step: 3500, Loss: 10.935732
Ste

In [8]:
sess.close()

In [9]:
# Calculate the error value for test image
# Read image data
fpI_test = open('/home2/bamboo/local_workspace/workspace/Out/n1/ewpLH-ImgRes_0.dig', 'rb')
nxI_test = fpI_test.read(4)
nxI_test = st.unpack("i", nxI_test)
nxI_test = nxI_test[0]
nyI_test = fpI_test.read(4)
nyI_test = st.unpack("i", nyI_test)
nyI_test = nyI_test[0]
img_target_test = []
for _ in range(nxI_test * nyI_test):
    data = fpI_test.read(8)
    data = st.unpack("d", data)
    data = data[0]
    img_target_test.append(data)
fpI_test.close()
img_target_test = np.array(img_target_test)
target_test = img_target_test.reshape(nyI_test, nxI_test)
fpIL_test = open('/home2/bamboo/local_workspace/workspace/Out/n1/ewpLH-LImgRes_0.dig', 'rb')
nxIL_test = fpIL_test.read(4)
nxIL_test = st.unpack("i", nxIL_test)
nxIL_test = nxIL_test[0]
nyIL_test = fpIL_test.read(4)
nyIL_test = st.unpack("i", nyIL_test)
nyIL_test = nyIL_test[0]
img_low_test = []
for _ in range(nxIL_test * nyIL_test):
    data = fpIL_test.read(8)
    data = st.unpack("d", data)
    data = data[0]
    img_low_test.append(data)
fpIL_test.close()
img_low_test = np.array(img_low_test)
low_test = img_low_test.reshape(nyIL_test, nxIL_test)

In [10]:
# Make dataset
height_test = nyI_test
width_test = nxI_test
num_blocks_test = (width_test - (wid_block - num_target)) * (height_test - (hei_block - 1))
init_target_test = 2 * height_test + 2
test_arr = []
for y in range(height_test - hei_block + 1):
    for x in range(0, width_test - wid_block + 1, 2):
        target_block_test = target_test[y:y+hei_target_block, x:x+wid_target_block]
        target_block_test = target_block_test.reshape(dim_target_block, 1)
        test_arr.extend(target_block_test)
        low_block_test = low_test[y + 2, x:x+wid_low_block]
        low_block_test = low_block_test.reshape(dim_low_block, 1)
        test_arr.extend(low_block_test)
test_data = np.array(test_arr)
test_x = test_data.reshape(int(test_data.size / dim_block), dim_block)

d = []
for y in range(hei_block - 1, height_test, 1):
    for x in range(half_widblock - 1, width_test - (half_widblock - 1), 2):
        label = target_test[y, x:x+2]
        d.append(label)
label = np.array(d)
test_label = np.reshape(label, (len(test_x), 2))

In [23]:
def relu(x):
    if x > 0:
        return x
    else:
        return 0
    
neuralReLU = np.frompyfunc(relu, 1, 1)

# Calculate 1st hidden layer
hidden_1 = neuralReLU(np.dot(test_x, weight_1) + bias_1)

# Caluclate 2nd hidden layer
hidden_2 = neuralReLU(np.dot(hidden_1, weight_2) + bias_2)

# Calculate output layer
out = neuralReLU(np.dot(hidden_2, weight_out) + bias_out)

In [39]:
# Calculate error value
def sumOfAbsLog(arr):
    sum = 0
    for i in arr:
        sum += np.log2(np.abs(i) + 1)
    return sum
    
def sumOfSquErr(arr):
    sum = 0
    for i in arr:
        sum += np.power(i, 2)
    return sum

# Sum of absolute log value
error_arr = out - test_label
N = error_arr[:,0].size


error_log_odd = sumOfAbsLog(error_arr[:,0]) / N
error_log_even = sumOfAbsLog(error_arr[:,1]) / N

# Root of Mean Squared Error
error_rmse_odd = np.sqrt(sumOfSquErr(error_arr[:,0]) / N)
error_rmse_even = np.sqrt(sumOfSquErr(error_arr[:,1]) / N)

print(error_log_odd)
print(error_log_even)
print(error_rmse_odd)
print(error_rmse_even)

2.6178759421296935
2.4014211948697026
17.722539228506097
17.504700958254755


In [38]:
PSNR_odd = 20*np.log10(255/error_rmse_odd)
PSNR_even = 20*np.log10(255/error_rmse_even)
print(PSNR_odd)
print(PSNR_even)

23.160284682123663
23.267709690878753


In [61]:
# Calculate error value w/o prediction
LImgRes = []
for y in range(hei_block - 1, height_test, 1):
    for x in range(half_widblock - 1, width_test - (half_widblock - 1), 2):
        label = low_test[y, x:x+2]
        LImgRes.append(label)
label = np.array(LImgRes)
LImgRes_label = np.reshape(label, (len(test_x), 2))

# Sum of absolute log value
error_arr_wo_prediction = LImgRes_label - test_label
N = error_arr_wo_prediction[:,0].size
error_log_odd_wo_prediction = sumOfAbsLog(error_arr_wo_prediction[:,0]) / N
error_log_even_wo_prediction = sumOfAbsLog(error_arr_wo_prediction[:,1]) / N

# Root of Mean Squared Error
error_rmse_odd_wo_prediction = np.sqrt(sumOfSquErr(error_arr_wo_prediction[:,0]) / N)
error_rmse_even_wo_prediction = np.sqrt(sumOfSquErr(error_arr_wo_prediction[:,1]) / N)

print(error_log_odd_wo_prediction)
print(error_log_even_wo_prediction)
print(error_rmse_odd_wo_prediction)
print(error_rmse_even_wo_prediction)

1.1021917040646363
1.6405694399701571
3.6927903201852086
8.243116113957182


In [62]:
PSNR_odd = 20*np.log10(255/error_rmse_odd_wo_prediction)
PSNR_even = 20*np.log10(255/error_rmse_even_wo_prediction)
print(PSNR_odd)
print(PSNR_even)

36.783710633952474
29.8089752600018
